# Cluebot - Modeling - XGBoost

In [44]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix

In [45]:
import pandas as pd

train_data = pd.read_csv('/Users/robin/Documents/GitHub/Cluebot/train_data.csv')
test_data = pd.read_csv('/Users/robin/Documents/GitHub/Cluebot/test_data.csv')

train_data.sample(5)

,EditType,EditID,comment,user,user_edit_count,user_distinct_pages,user_warns,user_reg_time,prev_user,common,...,num_recent_edits,num_recent_reversions,current_minor,current_timestamp,added_lines,previous_timestamp,deleted_lines,isvandalism,num_edits_5d_before,is_person
19412,change,234153304,NaN,12.207.12.225,196,121,8,20080825154259,J.delanoy,NaN,...,0,0,False,1219678979,"""* [[Anal Sex|Hollister Co.]], a clothing bran...",1219678744,"""* [[Hollister Co.]], a clothing brand from Ab...",True,2,0
13632,change,230728538,NaN,65.9.158.254,1,1,0,20080809012610,Arbor to SJ,NaN,...,0,0,False,1218245170,"""'''William Leonard Roberts II''' (born [[Janu...",1218240003,"""'''William Leonard Roberts II''' (born [[Janu...",True,24,1
3380,change,327246440,"/* 8. ""Give Me a Beat"" - 4:12 */",112.203.35.241,18,2,0,20091122063458,112.203.35.241,NaN,...,0,0,False,1258871698,"""* 2:21 - 3:08 [[Tom Petty and the Heartbreake...",1258822698,"""* 2:21 - 3:08 [[Tom Petty]] - """"[[American Gi...",False,14,0
12020,change,235511747,/* Crowley and modern occultism */,24.18.30.69,7,2,4,20080901032532,Discospinster,NaN,...,0,0,False,1220239532,"""Over the years, both became so fascinated by ...",1220239417,"""Over the years, both became so fascinated by ...",True,12,0
21681,change,328255933,/* Production */,Nisavid,34,33,0,1237476364,68.58.218.88,NaN,...,0,0,True,1259351927,"""""""AWESOM-O"""" was written and directed by seri...",1259079475,"""""""AWESOM-O"""" was written and directed by seri...",False,1,0


In [ ]:
features = ['user_edit_count', 'user_warns', 'user_distinct_pages', 'num_recent_edits', 'num_recent_reversions', 'current_minor', \
            'num_edits_5d_before', 'is_person', \
            'is_IP', 'account_age', 'comment_empty', 'word_count_added', 'word_count_deleted', 'vandalism_score']

from feature_engineer import preprocessor

preprocessor.preprocessor(train_data)
train_data = train_data.reset_index(drop=True)
train_data.sample(5)

,EditType,EditID,comment,user,user_edit_count,user_distinct_pages,user_warns,user_reg_time,prev_user,common,...,previous_timestamp,deleted_lines,isvandalism,num_edits_5d_before,is_person,comment_empty,account_age,is_IP,word_count_added,word_count_deleted
7853,change,329602447,/* Literary work */,Star reborn,638,0,0,1223247447,Star reborn,NaN,...,1259896495,"""Bridges' poetry was privately printed in the ...",False,5,1,False,424,False,214,212
8260,change,328402468,Removed category [[:Category:UNAF competitions...,Djln,272831,121713,10,1130953933,Djln,NaN,...,1259425763,[[Category:UNAF competitions]],False,3,0,False,1486,False,0,3
21776,change,255221004,/* Main Theories */,209.68.98.82,71,28,7,20081201160921,AdjustShift,NaN,...,1228147659,NaN,True,16,0,False,1,True,5,1
14475,change,235918633,/* Physical description */,203.22.118.10,368,162,17,20080902232728,69.179.56.211,NaN,...,1220394695,"""The leopard seal is large and muscular, with ...",True,6,0,False,1,True,174,170
18964,change,232006715,NaN,96.232.105.232,3,1,1,20080815002347,JForget,NaN,...,1218759801,{{Otheruses4|the first game in the series|an o...,True,8,0,True,1,True,17,16


In [64]:
import random
import numpy as np
import re
from collections import defaultdict

# Do a cross validation split manually
total_indices = list(range(len(train_data))) # 
random.shuffle(total_indices)
fold = [[], [], [], [], []]
[fold[0], fold[1], fold[2], fold[3], fold[4]] = np.array_split(total_indices, 5)
for i in range(5):
    fold[i] = list(fold[i])

# Get the difference between two strings and return a set of words
def get_difference(s1, s2):
    w1 = set(re.sub(r"[^\w\s]", " ", str(s1)).lower().split())
    w2 = set(re.sub(r"[^\w\s]", " ", str(s2)).lower().split())
    return w1 - w2

# Get a list of word probabilities using edits with given indices
def get_word_prob(indices):
    # Count the number of appearances of each word in vandalism/constructive edits
    vandalism_words_count = defaultdict(int)
    constructive_words_count = defaultdict(int)
    
    for i in indices:
        for word in get_difference(str(train_data.loc[i, 'added_lines']), str(train_data.loc[i, 'deleted_lines'])):
            if train_data['isvandalism'][i] == True:
                vandalism_words_count[word] += 1
            else:
                constructive_words_count[word] += 1

    # Combine all unique words
    all_words = set(vandalism_words_count) | set(constructive_words_count)

    # Compute smoothed probabilities
    word_probs = {
        word: (vandalism_words_count[word] + 1) / 
              (vandalism_words_count[word] + constructive_words_count[word] + 2)
        for word in all_words
    }
    return word_probs

# Compute vandalism scores for edits with given indices using a given list of word probabilities 
def compute_vandalism_scores(word_probs, indices):
    for i in indices:
        words = get_difference(train_data['added_lines'][i], train_data['deleted_lines'][i])
        probs_of_words = [word_probs.get(w, 0.5) for w in words]

        product_p = 1
        product_1_minus_p = 1
        for p in probs_of_words:
            product_p *= p
            product_1_minus_p *= (1-p)
        
        score = product_p / (product_p + product_1_minus_p) if (product_p + product_1_minus_p) != 0 else 1
        train_data.loc[i, 'vandalism_score'] = score



In [65]:
import xgboost

In [70]:
# Cluebot method

accuracy_scores = []

for i in range(5):
    i_0 = i % 5
    i_1 = (i + 1) % 5
    i_2 = (i + 2) % 5
    i_3 = (i + 3) % 5
    i_4 = (i + 4) % 5

    word_probs = get_word_prob(fold[i_2] + fold[i_3])
    compute_vandalism_scores(word_probs, fold[i_0] + fold[i_1])

    word_probs = get_word_prob(fold[i_0] + fold[i_1] + fold[i_2] + fold[i_3])
    compute_vandalism_scores(word_probs, fold[i_4])

    xgb_reg = xgboost.XGBClassifier(max_depth= 4, min_child_weight= 2,
                    gamma= 3.141595241599972, subsample= 0.818585447494315,
                    colsample_bytree= 0.7643301466688923, colsample_bylevel= 0.9834643450676969,
                    reg_alpha=0.3358100247796204, learning_rate= 0.0805746634624542, n_estimators= 916)
    xgb_reg.fit(train_data.loc[fold[i_0] + fold[i_1], features], train_data.loc[fold[i_0] + fold[i_1], 'isvandalism'])
    pred = xgb_reg.predict(train_data.loc[fold[i_4], features])

    accuracy_scores.append(np.sum(pred == train_data.loc[fold[i_4], 'isvandalism'])/len(train_data.loc[fold[i_4], 'isvandalism']))
    # print(confusion_matrix(train_data.loc[fold[i_4], 'isvandalism'], pred))

print(np.average(accuracy_scores))

0.922998108057633


In [71]:
# Original method

accuracy_scores = []

for i in range(5):
    i_0 = i % 5
    i_1 = (i + 1) % 5
    i_2 = (i + 2) % 5
    i_3 = (i + 3) % 5
    i_4 = (i + 4) % 5

    word_probs = get_word_prob(fold[i_0] + fold[i_1] + fold[i_2] + fold[i_3])
    compute_vandalism_scores(word_probs, fold[i_0] + fold[i_1] + fold[i_2] + fold[i_3] + fold[i_4])
    
    xgb_reg = xgboost.XGBClassifier(max_depth= 4, min_child_weight= 2,
                    gamma= 3.141595241599972, subsample= 0.818585447494315,
                    colsample_bytree= 0.7643301466688923, colsample_bylevel= 0.9834643450676969,
                    reg_alpha=0.3358100247796204, learning_rate= 0.0805746634624542, n_estimators= 916)
    xgb_reg.fit(train_data.loc[fold[i_0] + fold[i_1] + fold[i_2] + fold[i_3], features], train_data.loc[fold[i_0] + fold[i_1] + fold[i_2] + fold[i_3], 'isvandalism'])
    pred = xgb_reg.predict(train_data.loc[fold[i_4], features])

    accuracy_scores.append(np.sum(pred == train_data.loc[fold[i_4], 'isvandalism'])/len(train_data.loc[fold[i_4], 'isvandalism']))
    # print(confusion_matrix(train_data.loc[fold[i_4], 'isvandalism'], pred))

print(np.average(accuracy_scores))

0.915132940560994


In [72]:
# Modified Stacking Classifier Method

accuracy_scores = []

for i in range(5):
    i_0 = i % 5
    i_1 = (i + 1) % 5
    i_2 = (i + 2) % 5
    i_3 = (i + 3) % 5
    i_4 = (i + 4) % 5

    word_probs = get_word_prob(fold[i_1] + fold[i_2] + fold[i_3])
    compute_vandalism_scores(word_probs, fold[i_0])

    word_probs = get_word_prob(fold[i_0] + fold[i_2] + fold[i_3])
    compute_vandalism_scores(word_probs, fold[i_1])

    word_probs = get_word_prob(fold[i_0] + fold[i_1] + fold[i_3])
    compute_vandalism_scores(word_probs, fold[i_2])

    word_probs = get_word_prob(fold[i_0] + fold[i_1] + fold[i_2])
    compute_vandalism_scores(word_probs, fold[i_3])

    word_probs = get_word_prob(fold[i_0] + fold[i_1] + fold[i_2] + fold[i_3])
    compute_vandalism_scores(word_probs, fold[i_4])

    xgb_reg = xgboost.XGBClassifier(max_depth= 4, min_child_weight= 2,
                    gamma= 3.141595241599972, subsample= 0.818585447494315,
                    colsample_bytree= 0.7643301466688923, colsample_bylevel= 0.9834643450676969,
                    reg_alpha=0.3358100247796204, learning_rate= 0.0805746634624542, n_estimators= 916)
    xgb_reg.fit(train_data.loc[fold[i_0] + fold[i_1] + fold[i_2] + fold[i_3], features], train_data.loc[fold[i_0] + fold[i_1] + fold[i_2] + fold[i_3], 'isvandalism'])
    pred = xgb_reg.predict(train_data.loc[fold[i_4], features])

    accuracy_scores.append(np.sum(pred == train_data.loc[fold[i_4], 'isvandalism'])/len(train_data.loc[fold[i_4], 'isvandalism']))
    print(confusion_matrix(train_data.loc[fold[i_4], 'isvandalism'], pred))

print(np.average(accuracy_scores))


[[2432  191]
 [ 212 2250]]
[[2397  203]
 [ 183 2303]]
[[2399  187]
 [ 188 2312]]
[[2481  177]
 [ 203 2225]]
[[2412  169]
 [ 206 2298]]
0.9245319308290714
